In [1]:
import pandas as pd
from scipy.special import rel_entr
import numpy as np
import requests
from bs4 import BeautifulSoup
import string
import pyarrow.parquet as pq
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag

stopwords = set(stopwords.words('english'))

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')

In [2]:
from pymongo import MongoClient
import datetime

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']

west_virginia_collection = db['sdsc-west-virginia']
utah_collection = db['sdsc-utah']
montana_collection = db['sdsc-montana']

In [44]:
def find_keywords(text):
    tokens = word_tokenize(period_doc)
    manual_filter = ['’','”','“','—']

    filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_tokens = [token for token in filtered_tokens if token not in string.punctuation]
    filtered_tokens = [token for token in filtered_tokens if token not in manual_filter]

    tagged_tokens = pos_tag(filtered_tokens)

    desired_pos = ['NN', 'NNS', 'NNP', 'NNPS']

    keywords = [token for token, pos in tagged_tokens if pos in desired_pos]
    
    return keywords

In [19]:
start_date = pd.to_datetime('2019-12-18')
end_date = pd.to_datetime('2020-12-19')
period = df[(df['collectiondate'] >= start_date) & (df['collectiondate'] <= end_date)][:50]
period_doc = ' '.join(testdf['news'].astype(str))
period_keywords = find_keywords(period_doc)
top_period_keywords = {x[0] : x[1]/len(period_doc) for x in Counter(period_keywords).most_common(5)}

In [60]:
period_keywords = {x[0] : x[1]/len(period_doc) for x in Counter(keywords).most_common(5)}

In [61]:
out = []
for look_date in ['2019-12-18', '2019-12-19']:
    day_df = df[df['collectiondate'] == pd.to_datetime(look_date)][:50]
    day_doc = ' '.join(day_df['news'].astype(str))
    day_keywords = find_keywords(day_doc)
    day_counter = Counter(day_keywords)
    out.append({x : day_counter[x]/len(day_doc) for x in period_keywords})

In [70]:
for dists in out:
    print(sum(rel_entr(list(period_keywords.values()),list(dists.values()))))

0.0
0.0007404775326340417
